<a href="https://colab.research.google.com/github/jkm2000korea/master_degree/blob/main/6%EC%A3%BC%EC%B0%A8_Logistic_Regression_20240408.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/데이터마이닝_2024/예제 데이터')
import pandas as pd
import numpy as np

Mounted at /content/drive


In [ ]:
bank = pd.read_csv('UniversalBank.csv')
bank

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [ ]:
bank.drop(columns=['ID', 'ZIP Code'], inplace=True)   #ID, ZIP Code 변수 제거, 원본데이터셋에 반영
bank.columns = [c.replace(' ', '_') for c in bank.columns]   #변수 내 공백을 _로 변환
bank

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,1,1.9,3,0,0,0,0,1,0
4996,30,4,15,4,0.4,1,85,0,0,0,1,0
4997,63,39,24,2,0.3,3,0,0,0,0,0,0
4998,65,40,49,3,0.5,2,0,0,0,0,1,0


In [ ]:
bank['Education'] = bank['Education'].astype('category')   #Education 변수의 값을 범주형 값으로 인식할 수 있도록 변환
new_categories = {1: 'Undergrad', 2: 'Graduate', 3: 'Advanced/Professional'}   #Education 변수의 값에 따라 다음과 같이 분류하도록 셋팅
bank['Education'] = bank['Education'].cat.rename_categories(new_categories)   #Education 변수의 값을 'new_categories' 셋팅 기반으로 분류
bank = pd.get_dummies(bank, prefix_sep='_', drop_first=True)   #더미 변수 생성
bank

,Age,Experience,Income,Family,CCAvg,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard,Education_Graduate,Education_Advanced/Professional
0,25,1,49,4,1.6,0,0,1,0,0,0,False,False
1,45,19,34,3,1.5,0,0,1,0,0,0,False,False
2,39,15,11,1,1.0,0,0,0,0,0,0,False,False
3,35,9,100,1,2.7,0,0,0,0,0,0,True,False
4,35,8,45,4,1.0,0,0,0,0,0,1,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,1,1.9,0,0,0,0,1,0,False,True
4996,30,4,15,4,0.4,85,0,0,0,1,0,False,False
4997,63,39,24,2,0.3,0,0,0,0,0,0,False,True
4998,65,40,49,3,0.5,0,0,0,0,1,0,True,False


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

y = bank['Personal_Loan']
X = bank.drop(columns=['Personal_Loan'])
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

logit_reg = LogisticRegression()
logit_reg.fit(train_X, train_y)   #학습데이터로 모델 생성

print('intercept', logit_reg.intercept_[0])
print(pd.DataFrame({'coeff': logit_reg.coef_[0]}, index=X.columns).transpose())

intercept -0.3224382197048062
            Age  Experience    Income    Family     CCAvg  Mortgage  \
coeff -0.444509    0.432958  0.049696  0.634305  0.193296  0.000675   

       Securities_Account  CD_Account    Online  CreditCard  \
coeff            0.101774    2.552974 -0.459606   -0.418171   

       Education_Graduate  Education_Advanced/Professional  
coeff             2.97961                         3.525128  


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
pred = logit_reg.predict(valid_X)   #검증데이터로 예측
pred_prob = logit_reg.predict_proba(valid_X)   #predict_proba: 입력값(X)의 각 범주별 확률값 반환
result = pd.DataFrame({'actual': valid_y,
                       'p(0)': [p[0] for p in pred_prob],
                       'p(1)': [p[1] for p in pred_prob],
                       'predicted': pred})

case = [702, 932, 2764, 2721]
print(result.loc[case])

      actual      p(0)      p(1)  predicted
702        1  0.979978  0.020022          0
932        0  0.219119  0.780881          1
2764       0  0.967400  0.032600          0
2721       1  0.075379  0.924621          1


In [ ]:
#Multinomial Logistic Regression

In [ ]:
df1 = pd.read_csv('AccidentsFull.csv')
df1

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,SUR_COND,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR
0,0,2,2,1,0,1,0,3,0,0,...,4,0,3,1,1,1,1,0,0,1
1,1,2,1,0,0,1,1,3,2,0,...,4,0,3,2,2,0,0,1,0,0
2,1,2,1,0,0,1,0,3,2,0,...,4,1,2,2,2,0,0,1,0,0
3,1,2,1,1,0,0,0,3,2,0,...,4,1,2,2,1,0,0,1,0,0
4,1,1,1,0,0,1,0,3,2,0,...,4,0,2,3,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42178,0,2,1,0,1,1,0,1,0,0,...,1,2,1,1,1,0,0,1,0,0
42179,1,2,1,1,0,0,0,1,0,0,...,1,0,1,1,1,1,1,0,0,1
42180,0,2,2,0,0,1,0,1,0,0,...,1,0,1,1,1,0,0,1,0,0
42181,1,2,1,1,0,1,0,1,0,0,...,1,0,1,1,1,0,0,1,0,0


In [ ]:
import statsmodels.api as sm

X = df1[['ALCHL_I', 'WEATHER_R']]
y = df1['MAX_SEV_IR'].astype('category')

X = sm.add_constant(X)

model = sm.MNLogit(y, X)
result = model.fit()

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.743054
         Iterations 9
                          MNLogit Regression Results                          
Dep. Variable:             MAX_SEV_IR   No. Observations:                42183
Model:                        MNLogit   Df Residuals:                    42177
Method:                           MLE   Df Model:                            4
Date:                Sun, 14 Apr 2024   Pseudo R-squ.:                0.004253
Time:                        13:22:31   Log-Likelihood:                -31344.
converged:                       True   LL-Null:                       -31478.
Covariance Type:            nonrobust   LLR p-value:                 9.739e-57
MAX_SEV_IR=1       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.9947      0.075     13.257      0.000       0.848       1.142
ALCHL_I         -0.3670

In [ ]:
# 학습된 모델을 사용하여 예측 확률을 계산
pred_probs = result.predict(X)

print(pred_probs)

              0         1         2
0      0.490643  0.498994  0.010363
1      0.553473  0.441134  0.005393
2      0.553473  0.441134  0.005393
3      0.490643  0.498994  0.010363
4      0.394180  0.578669  0.027151
...         ...       ...       ...
42178  0.490643  0.498994  0.010363
42179  0.490643  0.498994  0.010363
42180  0.490643  0.498994  0.010363
42181  0.490643  0.498994  0.010363
42182  0.490643  0.498994  0.010363

[42183 rows x 3 columns]


In [ ]:
pred_probs_df = pd.DataFrame(pred_probs)

# 각 관측치에 대한 예측된 클래스 인덱스 찾기
predicted_classes = np.argmax(pred_probs_df.values, axis=1)

pred_probs_df['predicted_class'] = predicted_classes

print(pred_probs_df)

              0         1         2  predicted_class
0      0.490643  0.498994  0.010363                1
1      0.553473  0.441134  0.005393                0
2      0.553473  0.441134  0.005393                0
3      0.490643  0.498994  0.010363                1
4      0.394180  0.578669  0.027151                1
...         ...       ...       ...              ...
42178  0.490643  0.498994  0.010363                1
42179  0.490643  0.498994  0.010363                1
42180  0.490643  0.498994  0.010363                1
42181  0.490643  0.498994  0.010363                1
42182  0.490643  0.498994  0.010363                1

[42183 rows x 4 columns]


In [ ]:
###Ordinal Logistic Regression

In [ ]:
pip install mord

  Preparing metadata (setup.py) ... done
  Created wheel for mord: filename=mord-0.7-py3-none-any.whl size=9885 sha256=bf53eca6898d53fcc9151d565f25ce5bdc9efe0cc0086df09c60141de05f085e
  Stored in directory: /root/.cache/pip/wheels/77/00/19/3cea86fbfc737ec4acb515cd94497dcc33f943fa157548b96c
Successfully built mord


In [ ]:
df2 = pd.read_csv('AccidentsFull.csv')
df2

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,SUR_COND,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR
0,0,2,2,1,0,1,0,3,0,0,...,4,0,3,1,1,1,1,0,0,1
1,1,2,1,0,0,1,1,3,2,0,...,4,0,3,2,2,0,0,1,0,0
2,1,2,1,0,0,1,0,3,2,0,...,4,1,2,2,2,0,0,1,0,0
3,1,2,1,1,0,0,0,3,2,0,...,4,1,2,2,1,0,0,1,0,0
4,1,1,1,0,0,1,0,3,2,0,...,4,0,2,3,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42178,0,2,1,0,1,1,0,1,0,0,...,1,2,1,1,1,0,0,1,0,0
42179,1,2,1,1,0,0,0,1,0,0,...,1,0,1,1,1,1,1,0,0,1
42180,0,2,2,0,0,1,0,1,0,0,...,1,0,1,1,1,0,0,1,0,0
42181,1,2,1,1,0,1,0,1,0,0,...,1,0,1,1,1,0,0,1,0,0


In [ ]:
X = df2[['ALCHL_I', 'WEATHER_R']]
y = df2['MAX_SEV_IR']

In [ ]:
from mord import LogisticIT

logit = LogisticIT()   #서열형 로지스틱 회귀모델 생성 / IT: immediate thresholds
logit.fit(X,y)

print('theta', logit.theta_)   #LogisticIT에서 theta_는 절편을 대체
print('coefficients', -(logit.coef_))   #모델에서 -계수 값을 반환
print()

probs = logit.predict_proba(X)
results = pd.DataFrame({
    'actual': y, 'predicted': logit.predict(X),
    'p(0)': [p[0] for p in probs],
    'p(1)': [p[1] for p in probs],
    'p(2)': [p[2] for p in probs]
})
print(results)

theta [-1.06805982  2.77551018]
coefficients [0.40065291 0.25156044]

       actual  predicted      p(0)      p(1)      p(2)
0           1          1  0.496202  0.482516  0.021282
1           0          0  0.558818  0.424554  0.016627
2           0          0  0.558818  0.424554  0.016627
3           0          1  0.496202  0.482516  0.021282
4           0          1  0.397511  0.571049  0.031440
...       ...        ...       ...       ...       ...
42178       0          1  0.496202  0.482516  0.021282
42179       1          1  0.496202  0.482516  0.021282
42180       0          1  0.496202  0.482516  0.021282
42181       0          1  0.496202  0.482516  0.021282
42182       0          1  0.496202  0.482516  0.021282

[42183 rows x 5 columns]
